In [ ]:
import numpy as np
import flammkuchen as fl
import napari
from pathlib import Path

from split_dataset import SplitDataset
import json

import tifffile as tiff
import flammkuchen as fl 

In [ ]:
master =  Path(r"Z:\Hagar and Ot\e0075\habenula")
fish_list = list(master.glob("*_f*"))
fish = fish_list[0]

In [ ]:
anat = tiff.imread(fish / "anatomy_from_suite2p.tif")

In [ ]:
viewer = napari.view_image(anat,  colormap="gray")
lhab = np.zeros(np.shape(anat), dtype=int)
rhab = np.zeros(np.shape(anat), dtype=int)

lhab_layer = viewer.add_labels(lhab, name='left')
rhab_layer = viewer.add_labels(rhab, name='right')

In [ ]:
#### Run the next cells only after marking the regions you want to keep in napari

In [ ]:
labels_lhab = viewer.layers["left"].data
labels_rhab = viewer.layers["right"].data

In [ ]:
d = {
    'lhab_mask': labels_lhab,
    'rhab_mask': labels_rhab,
}
fl.save(fish / 'habenula_masks.h5', d)

In [ ]:
### go over each plane and keep coords within mask

In [ ]:
for fish in fish_list:
    print(fish)
    planes_path = fish / "suite2p"
    planes = list(planes_path.glob("*00*"))
    
    #loading the mask:
    labels_l = fl.load(fish / 'habenula_masks.h5')['lhab_mask']
    labels_r = fl.load(fish / 'habenula_masks.h5')['rhab_mask']

    count = 0
    for plane in planes:

        curr_mask_l = labels_l[count]
        curr_mask_r = labels_r[count]
        try: 
            suite2p_data = fl.load(plane / 'data_from_suite2p_cells.h5')
        except: 
            suite2p_data = fl.load(plane / 'data_from_suite2p_unfiltered.h5')
        all_coords = suite2p_data['coords']

        coords_lhab = []
        coords_rhab = []
        num_cells = np.shape(all_coords)[0]
        for cell in range(num_cells):
            tmp_cell_l = curr_mask_l[int(all_coords[cell, 1]), int(all_coords[cell, 2])]
            tmp_cell_r = curr_mask_r[int(all_coords[cell, 1]), int(all_coords[cell, 2])]
            
            if tmp_cell_l > 0:
                coords_lhab = coords_lhab + [cell]
            elif tmp_cell_r > 0:
                coords_rhab = coords_rhab + [cell]

        count += 1
        
        d = {
            'lhab_coords': coords_lhab,
            'rhab_coords': coords_rhab,
        }
        fl.save(plane / 'habenula_coords.h5', d)